# *Bonus* Assignment 9-2: Envelope tracking through a Ring with space charge kick implementation

<html>
    <div class="alert alert-info" role="alert" style="margin-top: 10px">
        <ul>
            <li>Implement the space charge kick map.</li>
            <li>Track the beam through a FODO cell with the kick map.</li>
            <li>**This is a bonus assignment without master solution. Please feel free to play with it and give feedback!** </li>
            </ul>
    </div>
</html>

<html>
    <div class="alert alert-info" style="background-color:rgba(255, 0, 0, 0.6);
                                         margin-top:10px;
                                         color:white;
                                         border-color:rgba(255, 0, 0, 0.3)">
        <strong>If you use google colab, run this cell:</strong>
    </div>
</html>

<html>
    <div class="alert alert-info" style="background-color:rgba(255, 0, 0, 0.6);
                                         margin-top:10px;
                                         color:white;
                                         border-color:rgba(255, 0, 0, 0.3)">
<strong>If you run it locally, run</strong>
               </div>
</html>

```bash
$ cd .../pam1-hs2021
...pam1-hs2021$ git pull
```
<html>
    <div class="alert alert-info" style="background-color:rgba(255, 0, 0, 0.6);
                                         margin-top:10px;
                                         color:white;
                                         border-color:rgba(255, 0, 0, 0.3)">
to get the updated repository.</div></html>

## Initialization
<html>
    <div class="alert alert-info" role="alert" style="margin-top: 10px">
        <ul>
            <li> Simply run this cell to instantiante an `Proton` species and set all global variables in the `Parameter` module. Take 7 MeV as the kinetic energy. </li>
        </ul>
    </div>
</html>

In [ ]:
import numpy as np
import AcceLEGOrator.Parameter as param
from AcceLEGOrator import Proton, Constants, Physics
from AcceLEGOrator import Gaussian, Bunch
import matplotlib.pyplot as plt
import copy

c = Constants.clight

Ekin = 7 # MeV

proton = Proton()

# set global variables
param.gamma_0 = Physics.getGamma(Ekin, proton.mass)
param.mass = proton.mass # MeV / c^2
param.charge = proton.charge # e

beta0 = Physics.getBeta(param.gamma_0)
p0 = param.gamma_0 * beta0 * param.mass   # MeV / c
Brho = p0 / param.charge * 1e6 / c # T*m

print('gamma =', param.gamma_0)
print('beta  =', beta0)
print('p0    =', p0 , 'MeV/c')
print('Br    =', Brho, 'Tm')

<html>
    <div class="alert alert-info" role="alert" style="margin-top: 10px">
        <ul>
            <li> Simply run this cell to define the test FODO with dipole cell.</li>
        </ul>
    </div>
</html>

In [ ]:
from AcceLEGOrator import Drift, Quadrupole, Dipole
# quadrupole length [m]
Lq = 0.5

# quandrupole strength [1/m^2]
kf = 0.1795
kd = -0.2071

# focusing quadrupole
# magnetic field gradient [T/m]
gradBf = kf * Brho
Qf = Quadrupole(Lq, gradBf)
Qf_h = Quadrupole(Lq/2., gradBf)
print('half focusing Quadrupole:', Qf_h)

# defocusing quadrupole
# magnetic field gradient [T/m]
gradBd = kd * Brho
Qd = Quadrupole(Lq, gradBd)
print('defocusing Quadrupole:', Qd)

# dipole
Ldi = 3.5
theta = 10*np.pi/180 # rad
# bending radius [m]
rho = Ldi / theta
# magnetic field [T]
B = Brho / rho
Di = Dipole(Ldi, B)

# drift
# drift length [m]
Ldr = 2.5
Dr = Drift(Ldr)
print('drift:', Dr)

# define FODO cell
fodo = [Qf_h, Dr, Di, Dr, Qd, Dr, Di, Dr, Qf_h]

<html>
    <div class="alert alert-info" style="background-color:rgba(255, 0, 0, 0.6);
                                         margin-top:10px;
                                         color:white;
                                         border-color:rgba(255, 0, 0, 0.3)">
        <strong>TODO:</strong>
        <ul>
            <li>Use the `getMatchedSigma` function in `Tracking.py` to calculate the the sigma matrix that is matched to the fodo cell. We assume a transverse emittance of 1/$\beta\gamma$ mm mrad in both planes. We set $\sigma_z$ = 1cm. </li>
        </ul>
    </div>
</html>

In [ ]:
from AcceLEGOrator import Tracking

emit = 1/(beta0*param.gamma_0)*1e-6 # m*rad
sigma_matched = Tracking.getMatchedSigma(cell=fodo, ex=emit, ey=emit, sigma_z=0.01)

<html>
    <div class="alert alert-info" style="background-color:rgba(255, 0, 0, 0.6);
                                         margin-top:10px;
                                         color:white;
                                         border-color:rgba(255, 0, 0, 0.3)">
        <strong>TODO:</strong>
           <ul>
            <li>Use the `trackTwiss` function in `Tracking.py` to track the twiss parameters for $n_{cells}=1$. </li>
            <li>Plot $\beta_x$ and $\beta_y$. The parameter `n_slice` cuts each element (except drift) into shorter slices to get a smoother output. Change $n_{slice}$ from 1 and see the difference. The result should be quite ok for $n_{slice}> 5$.</li>
            <li>Check if you can reproduce the following plot.</li>
        </ul>
    </div>
</html>

![fodo_a9](img/fodo_a9.png)

In [ ]:
twiss, lengths = Tracking.trackTwiss(...)
# plot
fig = plt.figure()
fig.set_size_inches(9,6)
axis = fig.add_subplot(111)
axis.set_xlabel('s [m]')
axis.set_ylabel(r'$\beta_x, \beta_y [m]$')

axis.plot(..., label=r'$\beta_x [m]$', linestyle='-', color='black')
axis.plot(..., label=r'$\beta_y [m]$', linestyle='-.', color='black')
axis.set_ylim([0,30])
axis.set_xlim([0,18])
axis.legend(loc='best')
plt.title(r'$\beta$ functions in one cell')
plt.show()

## Space charge kick implementation

In the last assignment we have combined space charge to each map (drift and quadrupole) to define a NEW map, and we only consider the 2 horizontal dimensions. Also we have direcly taken the formula for the uniform beam and assumed the beam area for a Gaussain beam. In paper [1], the authors have derived a full analytic space charge model for Gaussian
beams. Let's take their result and see if we can improve our simulation.

According to Eq. (3.5) of [1], the space charge kick of a Gaussian beam can be considered as a defocusing quadrupole in both planes with inverse focal lengths
![sc_quadrupole](img/sc_quadrupole.png)
where
- $(x_1, x_3) = (x, y)$, and $\sigma_{11}=\langle x^2\rangle=\Sigma[0,0], \sigma_{33}=\langle y^2\rangle=\Sigma[2,2], \sigma_{13}=\langle xy\rangle=\Sigma[0,2]$
- $$\tilde{K} = K\cdot dl = \frac{2N \cdot \frac{e^2}{4\pi \epsilon_0 m_0 c^2}}{\sqrt{2\pi}\sigma_z \beta^2\gamma^3}\cdot dl$$
where N is the number of particles in the bunch, $\sigma_z$ is the bunch length, $dl$ the length of each slice over which the space charge force is effective (Eq. (2.9) of [1]).
- The value of $a$ and $b$ come from Eq. (2.5) of [1]
![ab](img/ab.png)
You can start from calculating $z_1 - z_2$.

<html>
    <div class="alert alert-info" style="background-color:rgba(255, 0, 0, 0.6);
                                         margin-top:10px;
                                         color:white;
                                         border-color:rgba(255, 0, 0, 0.3)">
        <strong>TODO:</strong>
        <ul>
            <li> Complete the following function to get the focal lengths of space charge kick.</li>
        </ul>
    </div>
</html>

In [ ]:
def get_finv(sigma, N, particle, dl):
    beta_0 = Physics.getBeta(param.gamma_0)
    eps0 = Constants.epsilon0
    c    = Constants.clight
    e    = Constants.echarge
    q    = particle.charge * e
    
    sigma_z = np.sqrt(sigma[4,4])
    ...
    
    K = ...
    K_tilde = K * dl

    fx_inv = K_tilde * ...
    fy_inv = K_tilde * ...
    return fx_inv, fy_inv

<html>
    <div class="alert alert-info" style="background-color:rgba(255, 0, 0, 0.6);
                                         margin-top:10px;
                                         color:white;
                                         border-color:rgba(255, 0, 0, 0.3)">
        <strong>TODO:</strong>
        <ul>
            <li>Define the space charge kick map.</li>
        </ul>
    </div>
</html>

In [ ]:
from AcceLEGOrator import Map

class SpaceChargeKick(Map):
    
    # param length_eff: effective length [m]
    # param sigma: the sigma matrix of the bunch being tracked
    # param N: number of particles in the bunch
    # param particle: type of paticle
    def __init__(self, length_eff, sigma, N, particle):
        self.length = 0 # real length of the kick is zero!
        self.length_eff = length_eff # length of the element it attches to
        self.__sigma = sigma
        self.__N = N
        self.__particle = particle
        
        fx_inv, fy_inv = ...
        
        R = np.real_if_close(np.matrix([[1, 0, 0, 0, 0, 0],
                      [fx_inv, 1, 0, 0, 0, 0],
                      [0, 0, 1, 0, 0, 0],
                      [0, 0, fy_inv, 1, 0, 0],
                      [0, 0, 0, 0, 1, 0],
                      [0, 0, 0, 0, 0, 1]], dtype = 'complex_'))
        super(SpaceChargeKick, self).__init__(R, self.length)
    
    
    def __str__(self):
        return 'SpaceChargeKick(N = ' + str(self.__N) + ' [A],\n'\
               + '\t\t      particle = ' + str(self.__particle) + ')\n'\
               + '\t\t      length_eff = ' + str(self.length_eff) + '[m]'
    
    def get(self, length, sigma=None):
        # return another SpaceChargeDrift with different sigma and/or length
        if sigma is None:
            return SpaceChargeDrift(length, self.__sigma, self.__N, self.__particle)
        else:
            return SpaceChargeDrift(length, sigma, self.__N, self.__particle)

## Tracking through space charge incorporated FODO cells
<html>
    <div class="alert alert-info" style="background-color:rgba(255, 0, 0, 0.6);
                                         margin-top:10px;
                                         color:white;
                                         border-color:rgba(255, 0, 0, 0.3)">
        <strong>TODO:</strong>
        <ul>
            <li>Define the following function to track sigma through one cell with space charge kick, then get the new cell.</li>
        </ul>
    </div>
</html>

#### Notes
- Since sigma is changing throughout tracking, we need to update sigma and the accelerator elements together. For example, to track sigma through [drift, quadrupole]
    - progagate sigma through first half drift;
    - collect the sigma after first half drift;
    - use the new sigma to define the kick;
    - propagate sigma through kick;
    - collect the sigma after kick;
    - use the new sigma to define the second half drift;
    - propagate sigma through the new second half drift;
    - collect the sigma after the new second half drift;
    - same procedure for quadrupole ...
  
  Note that the above process should loop over each **slice** of acclerator element. Each slice of M has length $M.length/n_{slice}$. For simplicity we can set $n_{slice}=1$ here.

In [ ]:
def track_fodo_sckick(cell, twiss_init, sigma_init, N, particle, n_cells=1, n_slice=1):
    # initial twiss
    # for x plane
    ax0, bx0, gx0, ay0, by0, gy0 = twiss_init
    # initial container of the twiss parameters
    twiss = [[ax0, bx0, gx0, ay0, by0, gy0]]
    Jx = np.matrix([[bx0, -ax0],
                    [-ax0, gx0]]) # submatrix in x plane
    Jy = np.matrix([[by0, -ay0],
                    [-ay0, gy0]]) # submatrix in y plane
    
    # initial sigma
    sigmas = np.expand_dims(sigma, axis=0)
    
    # initial length
    length_till_now = 0.
    lengths = [0,]

    for n in range(n_cells):
        for M in cell:
            L = M.length
            for i in range(n_slice):
                # use the current sigma to define slice i map
                
                # first half of slice i
                
                # calculate length
                
                # propagate twiss using M_slice_i_1
                
                # propagate sigma using M_slice_i_1
                
                # collect twiss
                
                # collect sigma
                
                
                # space charge kick
                
                # calculate length
                
                # propagate twiss using M_kick
                
                # propagate sigma using M_kick
                
                # collect twiss
                
                # collect sigma
                
                
                # second half of slice i
                
                # calculate length
                
                # propagate twiss using M_slice_i_2
                
                # propagate sigma using M_slice_i_2
                
                # collect twiss
                
                # collect sigma

    return twiss, sigmas, lengths

<html>
    <div class="alert alert-info" style="background-color:rgba(255, 0, 0, 0.6);
                                         margin-top:10px;
                                         color:white;
                                         border-color:rgba(255, 0, 0, 0.3)">
        <strong>TODO:</strong>
        <ul>
            <li>Track the Twiss parameters through the space charge incorprated FODO cell for different number of particles in bunch.</li>
        </ul>
    </div>
</html>

Note:
- Since the actual cell can only be obtained via tracking, you need to in principle track one cell and get the transfer map of this cell first, and then use the `getTwissInitial` function to get the matched Twiss parameters. After that, you can use this new set of twiss parameters as the initial twiss parameter to track again. 

In [ ]:
twiss_sc_init, sigmas_sc_init, lengths_sc_init = track_fodo_sckick(...)
twiss_sc, sigmas_sc, lengths_sc = track_fodo_sckick(..., twiss_init=twiss_sc_init[-1])
fig = plt.figure()
fig.set_size_inches(6,6)
axis = fig.add_subplot(111)
axis.set_xlabel('s [m]')
axis.set_ylabel(r'$\beta_x, \beta_y [m]$')

...

axis.legend(loc='upper left')
plt.show()

## References
- [1] M. Holz, V. Ziemann, Analytic Space-Charge Model for Gaussian Beams with cross-plane Coupling, https://uu.diva-portal.org/smash/get/diva2:1160961/FULLTEXT01.pdf